<a href="https://colab.research.google.com/github/ainfanzon/Cockroach_IAM_Workshop/blob/main/GCP_Colab_notebooks/Exercise_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://drive.google.com/uc?id=1XYr9Tyrz31a5kZdo601xD1QWz_YM8-H3">

### CockroachDB is a distributed SQL database that is __*highly scalable*__, __*resilient*__, and __*easy to use*__.

# Identity and Access Management Workshop.
---
## CockroachDB Overview.

In this Lab you will:

1. Start a three node cluster.
1. Verify the cluster deployment.
1. Load data and verify replication.

Assign your Public IP to the **my_ip** variable and your user id to the **uid** below:

In [ ]:
my_ip = '34.223.52.6'
uid = 'roachie'

---

## 1. Start a three node cluster.

<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
</style>
</head>
<body>

<table style="width:100%">
  <tr>
      <td align="right">
          <img src="https://drive.google.com/uc?id=1k_qixt3JcQTD13Zhi_jycORL046zrYR7" width="850" height="250">
      </td>
  </tr>
</table>

</body>
</html>

To start a cluster with three nodes execute the steps below:

- On your laptop open a terminal window and connect to the virtual machine using ssh (run the cell below to create the ssh command)

In [ ]:
display(Markdown(f"> ssh {uid}@{my_ip}"))

- To start a three nodes cluster execute the __**cockroach start**__ (see example below). For the lab there is a script (`strt_crdb.sh`) you can execute. The script is located in the `/home/cockroach/scripts/` directory.

> ```/home/cockroach/scripts/start_crdb.sh```

<p> The script executes the **cockroach start** command
<br><br>
&emsp;&emsp;cockroach start<br>
&emsp;&emsp;&emsp;&emsp;--insecure<br>
&emsp;&emsp;&emsp;&emsp;--listen-addr=&lt;ip address&gt;:&lt;sql listening port&gt;<br>
&emsp;&emsp;&emsp;&emsp;--join=&lt;ip address&gt;:&lt;sql listening port&gt;, ... ,&lt;ip address&gt;:&lt;sql listening port&gt;<br>
&emsp;&emsp;&emsp;&emsp;--http-addr=&lt;ip address&gt;:&lt;http listening port&gt;<br>
&emsp;&emsp;&emsp;&emsp;--locality=region=us-west,zone=us-west-1a<br>
&emsp;&emsp;&emsp;&emsp;--store=/home/cockroach/data/cr_data_1<br>
&emsp;&emsp;&emsp;&emsp;--background<br>
<br>
&emsp;&emsp;cockroach init --insecure --host &lt;ip address&gt;
</p>

You should see a `Cluster successfully initialized` message.
<br><br>



---

## 2. Verify the cluster deployment

On your terminal window verify there are three instances of the `cockroach` process running on different ports. The command below displays the `process id` and the full command used to launch the process.

> `pgrep -a cockroach`

&emsp;NOTE: Each process will be running on the same IP address but different ports. The command below displays the listneing address.

> ```pgrep -a cockroach | awk '{ printf "%s | %s\n", $5, $7}'```

> ```
--listen-addr=10.0.1.2:26257 | --http-addr=10.0.1.2:8080
--listen-addr=10.0.1.2:26258 | --http-addr=10.0.1.2:8081
--listen-addr=10.0.1.2:26259 | --http-addr=10.0.1.2:8082
```

- Execute the cell below and click the generated url to open another browser tab to display the cockroach **DB Console**:

In [ ]:
print(f"http://{my_ip}:8080/#/overview/list")

- Use SQL to display the same information


In [ ]:
import psycopg2
import pandas as pd

from IPython.display import IFrame, display, HTML, Markdown

pd.set_option('display.max_colwidth', None)

try:
    conn = psycopg2.connect(
        database = 'defaultdb'
      , user = 'root'
      , host = my_ip
      , port = '26257'
      , sslmode = 'disable'
    )
    display(Markdown("## Connection successful!"))
except psycopg2.OperationalError as e:
    print(f"Error connecting to database: {e}")

- Execute the SQL below to display the cluster:
    - Node id
    - Advertised Address
    - Version
    - Up Time
    - Number of Ranges
    - Number of Leaders
    - Server Status and,
    - Membership Status.

In [ ]:
try:
    cursor = conn.cursor()
    cursor.execute("""
        SELECT gn.node_id AS "Node ID"
             , gn.advertise_sql_address AS "Advertised Address"
             , gn.build_tag AS "Version"
             , current_timestamp() AT TIME ZONE 'UTC' - gn.started_at AS "Up Time"
             , "ranges" AS "Ranges"
             , leases AS "Leaders"
             , CASE WHEN is_live THEN 'LIVE' ELSE 'DEAD' END AS "Server Status"
             , gl.membership AS "Membership Status"
        FROM crdb_internal.gossip_nodes AS gn join crdb_internal.gossip_liveness AS gl USING(node_id)
        """)
    result_set = cursor.fetchall()
    df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
    df_result_set.set_index('Node ID', inplace=True)
    display(df_result_set)
    cursor.close()
except psycopg2.OperationalError as e:
    cursor.close()
    conn.commit()

### Few points to note.

Replicas are automatically distributed across cluster nodes.

- How many ranges does each replica (node) has?
- What is the advantage of using the private IP instead of the Public one?
- How many ranges are unavailable?
- Are all the nodes active and live?
<br><br>

---

## 3. Create and populate the IAM database.

Next step is to create and populate the Identity Access Management database.

<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
</style>
</head>
<body>

<table style="width:100%">
  <tr>
      <td align="right">
          <img src="https://drive.google.com/uc?id=1hhcjsCJ7TO7nhUmR2JRoBhi7BZ-L5SIh" width="550" height="400">
      </td>
  </tr>
</table>

</body>
</html>
<br>

Follow the steps below to create a database and load the data:

- On your laptop, open a second terminal window using ssh (see above).

- Change to the **/home/cockroach/dump** directory and execute the **Python** http server.

> ```
cd /home/cockroach/dump
python -m http.server 3000
```
&emsp;You should see the HTTP server is running on port 3000

> <code>
Serving HTTP on 0.0.0.0 port 3000 (http://0.0.0.0:3000/) ...
</code>

- Use the first terminal to execute the scripts to create the schema and populate the database.

    - First update the SQL script with your PRIVATE IP address. You can use the **bash** command below:<br>
<code>
sed -E -i s/HOST_IP/$(hostname -I | awk '{print $1}')/ /home/cockroach/sql/iam.sql
</code><br>

    - Then execute the SQL script<br>
```cockroach sql --host $(hostname -I) -u root -d default -f /home/cockroach/sql/iam.sql --insecure```

- Execute the cell below to compare the number of ranges and their distribution across the nodes in the cluster.

In [ ]:
try:
    cursor = conn.cursor()
    cursor.execute("""
        SELECT gn.node_id AS "Node ID"
             , gn.advertise_sql_address AS "Advertised Address"
             , gn.build_tag AS "Version"
             , current_timestamp() AT TIME ZONE 'UTC' - gn.started_at AS "Up Time"
             , "ranges" AS "Ranges"
             , leases AS "Leaders"
             , CASE WHEN is_live THEN 'LIVE' ELSE 'DEAD' END AS "status"
             , gl.membership
        FROM crdb_internal.gossip_nodes AS gn join crdb_internal.gossip_liveness AS gl USING(node_id)
        """)
    result_set = cursor.fetchall()
    df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
    df_result_set.set_index('Node ID', inplace=True)
    display(df_result_set)
    cursor.close()
except psycopg2.OperationalError as e:
    cursor.close()
    conn.commit()

### Few points to note.

- Why is there a difference in the number of ranges with first execution of the statement?
- Why is there a difference in the number of Leaders?
- Compare with the number of ranges in the DB Console.

---
## CockroachDB is a distributed SQL database that is __*highly scalable*__, __*resilient*__, and __*easy to use*__.
<img src="https://drive.google.com/uc?id=1XYr9Tyrz31a5kZdo601xD1QWz_YM8-H3">

---

# Appendix

Workshop CRDB user id and passowrd

> <p>uid = roachie</p>

List CRDB process id and process name.

> <code>pgrep -l cockroach</code>

List the listening address of each `cockroach` process.

> <code>pgrep -a cockroach | awk '{ print $5}'</code>

Kill ALL CRDB processes

> <code>kill -9  $(pgrep cockroach)</code>

Remove all CRDB files

> <code>rm -fR /home/cockroach/data/*</code>

Replace ip in file

> ```sed -E -i s/HOST_IP/$(hostname -I | awk '{print $1}')/ iam.sql```

Listening ports

> ```netstat -ntlp```